In [13]:
import tensorflow as tf
import pickle
import numpy as np
import timeit
import time
import load_cifar
from scipy.sparse import csr_matrix
from load_cifar import features_reshape
from sklearn.utils import shuffle
from tensorflow.contrib.layers import flatten

In [14]:
tf.reset_default_graph()

In [15]:
valid_features, valid_labels = pickle.load(open('vali_data.pickle','rb'))
training_features,training_labels = pickle.load(open('train_data.pickle','rb'))
test_features, test_labels = pickle.load(open('test_data.pickle','rb'))
print(training_labels.shape)
print(training_labels[0])
print(training_features.shape)

(40000, 10)
  (0, 1)	1.0
(40000, 3072)


In [16]:
valid_labels = np.squeeze(valid_labels.todense())
training_labels = np.squeeze(training_labels.todense())
test_labels = np.squeeze(test_labels.todense())

In [17]:
training_features = features_reshape(training_features)
valid_features = features_reshape(valid_features)
test_features = features_reshape(test_features)
print(training_features.shape)
print(valid_features.shape)
print(test_features.shape)

(40000, 32, 32, 3)
(10000, 32, 32, 3)
(10000, 32, 32, 3)


<h1>Hyperparameter</h1>

In [26]:
EPOCHS = 10
BATCH_SIZE = 32
lr = 0.001

<h1>Placeholder</h1>

In [19]:
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None, 10))

<h1>LeNet-5</h1>

In [20]:
def LeNet(x):    

    conv1_w = tf.Variable(tf.truncated_normal(shape = [5,5,3,6], mean = 0, stddev = 0.1))
    conv1_b = tf.Variable(tf.zeros(6))
    conv1 = tf.nn.conv2d(x,conv1_w, strides = [1,1,1,1], padding = 'VALID') + conv1_b 

    conv1 = tf.nn.relu(conv1)

    pool_1 = tf.nn.max_pool(conv1, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')
    
    conv2_w = tf.Variable(tf.truncated_normal(shape = [5,5,6,16], mean = 0, stddev = 0.1))
    conv2_b = tf.Variable(tf.zeros(16))
    conv2 = tf.nn.conv2d(pool_1, conv2_w, strides = [1,1,1,1], padding = 'VALID') + conv2_b

    conv2 = tf.nn.relu(conv2)

    pool_2 = tf.nn.max_pool(conv2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID') 
    
    fc1 = flatten(pool_2)
    
    fc1_w = tf.Variable(tf.truncated_normal(shape = (400,120), mean = 0, stddev = 0.1))
    fc1_b = tf.Variable(tf.zeros(120))
    fc1 = tf.matmul(fc1,fc1_w) + fc1_b
    
    fc1 = tf.nn.relu(fc1)

    fc2_w = tf.Variable(tf.truncated_normal(shape = (120,84), mean = 0, stddev = 0.1))
    fc2_b = tf.Variable(tf.zeros(84))
    fc2 = tf.matmul(fc1,fc2_w) + fc2_b

    fc2 = tf.nn.relu(fc2)
    
    fc3_w = tf.Variable(tf.truncated_normal(shape = (84,10), mean = 0 , stddev = 0.1))
    fc3_b = tf.Variable(tf.zeros(10))
    predicted_logits = tf.matmul(fc2, fc3_w) + fc3_b
    return predicted_logits

<h1>Cost and Optimization</h1>

In [21]:
predicted_logits = LeNet(x)
loss_operation = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = predicted_logits, labels = y))
optimizer = tf.train.AdamOptimizer(learning_rate = lr)
training_operation = optimizer.minimize(loss_operation)

In [22]:
correct_prediction = tf.equal(tf.argmax(predicted_logits, 1), tf.argmax(y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

<h1>Training, validation and testing</h1>
<h2>Train your model only 10 epochs.</h2>
<h2>1.Print out validation accuracy after each training epoch</h2>
<h2>2.Print out training time for each training epoch</h2>
<h2>3.Print out testing accuracy</h2>

In [27]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(training_features)
    for i in range(EPOCHS):
        start_time = time.time()
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = training_features[offset:end,:], training_labels[offset:end,:]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
        use_time = time.time() - start_time   
        validation_accuracy = sess.run(accuracy_operation,feed_dict={x:valid_features, y:valid_labels})
        print("epoch {0}, Validation Accuracy:{2:.3f}".format(i+1, use_time, validation_accuracy))

    test_accuracy = sess.run(accuracy_operation,feed_dict={x:test_features, y:test_labels})
    print("Test Accuracy: {:.3f}".format(test_accuracy))

epoch 1, Validation Accuracy:0.468
epoch 2, Validation Accuracy:0.514
epoch 3, Validation Accuracy:0.554
epoch 4, Validation Accuracy:0.577
epoch 5, Validation Accuracy:0.581
epoch 6, Validation Accuracy:0.587
epoch 7, Validation Accuracy:0.587
epoch 8, Validation Accuracy:0.590
epoch 9, Validation Accuracy:0.585
epoch 10, Validation Accuracy:0.586
Test Accuracy: 0.581
